In [1]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [2]:
'''
# Cargo de Google Drive los archivos

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_train_transaction='19setxTbNvfO5L5h-9h5-bcFcl7Ml1kHt'
downloaded = drive.CreateFile({'id': id_train_transaction})
downloaded.GetContentFile('train_transaction.csv')

id_train_identity = '1XUUs36t6K2ByA7xuQuTzRH_smXVtmVjF'
downloaded = drive.CreateFile({'id': id_train_identity})
downloaded.GetContentFile('train_identity.csv')

id_test_transaction='1LnLkTi7YYu95qlR6r7fDGbQUJrNR2uYE'
downloaded = drive.CreateFile({'id': id_test_transaction})
downloaded.GetContentFile('test_transaction.csv')

id_test_identity = '1Pro4HlgAnTo2UmGhS7NReITPAJxxPolA'
downloaded = drive.CreateFile({'id': id_test_identity})
downloaded.GetContentFile('test_identity.csv')'''

"\n# Cargo de Google Drive los archivos\n\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\nid_train_transaction='19setxTbNvfO5L5h-9h5-bcFcl7Ml1kHt'\ndownloaded = drive.CreateFile({'id': id_train_transaction})\ndownloaded.GetContentFile('train_transaction.csv')\n\nid_train_identity = '1XUUs36t6K2ByA7xuQuTzRH_smXVtmVjF'\ndownloaded = drive.CreateFile({'id': id_train_identity})\ndownloaded.GetContentFile('train_identity.csv')\n\nid_test_transaction='1LnLkTi7YYu95qlR6r7fDGbQUJrNR2uYE'\ndownloaded = drive.CreateFile({'id': id_test_transaction})\ndownloaded.GetContentFile('test_transaction.csv')\n\nid_test_identity = '1Pro4HlgAnTo2UmGhS7NReITPAJxxPolA'\ndownloaded = drive.CreateFile({'id': id_test_identity})\ndownloaded.GetContentFile('tes

In [3]:
train_identity = pd.read_csv('./ieee-fraud-detection/train_identity.csv')

train_transaction = pd.read_csv('./ieee-fraud-detection/train_transaction.csv')

# Analizo los datasets

Creo los datasets encodeardos, limpios y escalados que voy a utilizar para entrenar y validar el modelo.

In [4]:
train_identity

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,NaN,...,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,NaN,...,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,NaN,NaN,0.0,...,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [5]:
train_transaction

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> ***Observo que train_transaction tiene una columna de TransactionDT.***



Esta es una columna es de tipo datetime y representa el tiempo transcurrido desde un instante de referencia (desconocido). 

Esto significa que debe permanecer ordenada para evitar time-travel. 

In [6]:
# Verifico que esten ordenado los registros en el tiempo

list(train_transaction.TransactionDT) == sorted(train_transaction.TransactionDT, reverse=False)

True

## Busco el dataset completo con el que voy a trabajar

Uno los data sets de trainning que nos dan.

In [7]:
all_data = train_transaction.merge(train_identity, on='TransactionID', how='outer')

In [8]:
import gc

# Libero memoria del sistema
del(train_transaction)
del(train_identity)

# using the garbage collector to manually collect the garbage
gc.collect()

20

Este truco lo saque de Kaggle para administrar la memoria.

In [9]:
display(all_data)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Verifico que mantenga el orden segun TransactionDT

In [10]:
all_data.TransactionDT.hasnans

False

In [11]:
list(all_data.TransactionDT) == sorted(all_data.TransactionDT, reverse=False)

True

## Tomo 80 columnas del DataSet

Segun el modelo baseline de Regresion Logistica, tomo las 80 features mas importantes

In [12]:
top80features = ['C14',
 'C2',
 'C11',
 'C13',
 'C7',
 'C3',
 'V192',
 'V222',
 'V179',
 'DeviceType',
 'V207',
 'V221',
 'C5',
 'V73',
 'V259',
 'V48',
 'M7',
 'V227',
 'V267',
 'card6',
 'V269',
 'V189',
 'V49',
 'ProductCD',
 'V147',
 'M1',
 'V18',
 'V257',
 'V200',
 'id_15',
 'V50',
 'V171',
 'V149',
 'V139',
 'D8',
 'id_29',
 'V208',
 'V323',
 'C8',
 'V182',
 'V210',
 'D3',
 'M6',
 'V188',
 'V229',
 'V33',
 'V71',
 'V285',
 'V153',
 'V58',
 'id_17',
 'V44',
 'TransactionDT',
 'M8',
 'V268',
 'V29',
 'V246',
 'M4',
 'V13',
 'V255',
 'V174',
 'V195',
 'M3',
 'V82',
 'V21',
 'V185',
 'V203',
 'V83',
 'V184',
 'V176',
 'id_36',
 'V294',
 'V168',
 'V332',
 'V175',
 'V333',
 'V225',
 'id_35',
 'V292',
 'V326']

In [14]:
features_adicionales = ['id_31', 'id_33', 'DeviceType', 'DeviceInfo', 'isFraud', 'TransactionDT' ]

for feat in features_adicionales:
    if (not feat in top80features):
        top80features.append(feat)

In [15]:
all_data = all_data[top80features]

display(all_data)

,C14,C2,C11,C13,C7,C3,V192,V222,V179,DeviceType,...,V175,V333,V225,id_35,V292,V326,id_31,id_33,DeviceInfo,isFraud
0,1.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
1,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
2,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
3,1.0,5.0,1.0,25.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
4,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,mobile,...,0.0,0.0,0.0,T,1.0,0.0,samsung browser 6.2,2220x1080,SAMSUNG SM-G892A Build/NRD90M,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,2.0,1.0,1.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
590536,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
590537,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
590538,1.0,1.0,1.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0


## Extraigo los target y features del dataset y creo los sets de trainning y testing.

Creo `data_train` y `data_test` para el set de `data` y

`label_train`, `label_test` para el set de `target`.

> Nota: El target es `isFraud`

### Particiono la data como train_set y test_set

Tomo un 20% de la data como `test_set` y el restante para `train_set` del modelo.

Me baso en TransactionDT como separador de registros.

In [16]:
labels = all_data.isFraud

In [17]:
# Tomo el 80% 
ochenta_porciento = int(all_data['TransactionDT'].size * 0.8)

In [18]:
data_train = all_data.iloc[: ochenta_porciento]

data_validation = all_data.iloc[ochenta_porciento:]

In [19]:
label_train = labels.iloc[:ochenta_porciento]

label_validation = labels.iloc[ochenta_porciento:]

## Limpio la data

Para los features categoricos reemplazo los 'nans' por 'desconocido' y para los features numericos los reemplazo por su mean de cada columna.

> Nota: Para el validation test reemplazo los nans por los means correspondientes a las columnas del trainning_set.
>
>  No se vuelven a calcular los means por columnas.

In [20]:
# Busco los promedios de cada columna numerica para el trainning_set

mean_values={}

for i in data_train.columns:
  if (str(data_train[i].dtype) != 'object'):
    mean_values[i] = data_train[i].mean()

In [21]:
# Limpio la data

def limpiar_dataset(dataset, mean_values):
  
  for i in dataset.columns:

    if (str(dataset[i].dtype) != 'object'):
      dataset[i].fillna(mean_values[i], inplace=True)
    
    else:
      dataset[i].fillna('desconocido',inplace=True)

In [22]:
limpiar_dataset(data_train, mean_values)
display(data_train)

/home/paredesj/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,C14,C2,C11,C13,C7,C3,V192,V222,V179,DeviceType,...,V175,V333,V225,id_35,V292,V326,id_31,id_33,DeviceInfo,isFraud
0,1.0,1.0,2.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
1,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
2,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
3,1.0,5.0,1.0,25.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
4,1.0,1.0,1.0,1.0,0.0,0.0,1.00000,1.000000,0.000000,mobile,...,0.000000,0.000000,0.000000,T,1.0,0.000000,samsung browser 6.2,2220x1080,SAMSUNG SM-G892A Build/NRD90M,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,1.0,2.0,1.0,2.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,2.0,0.401879,desconocido,desconocido,desconocido,0
472428,1.0,2.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,2.0,0.401879,desconocido,desconocido,desconocido,0
472429,1.0,1.0,1.0,1.0,0.0,0.0,1.00000,1.000000,0.000000,desktop,...,0.000000,0.000000,0.000000,T,1.0,0.000000,chrome 65.0,1440x900,MacOS,0
472430,1.0,1.0,1.0,2.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0


In [23]:
data_validation

,C14,C2,C11,C13,C7,C3,V192,V222,V179,DeviceType,...,V175,V333,V225,id_35,V292,V326,id_31,id_33,DeviceInfo,isFraud
472432,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,desktop,...,0.0,NaN,0.0,F,1.0,NaN,chrome 65.0,NaN,NaN,1
472433,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,mobile,...,0.0,NaN,0.0,F,1.0,NaN,chrome generic for android,NaN,F3311,0
472434,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,desktop,...,0.0,NaN,0.0,F,1.0,NaN,chrome 65.0,NaN,Windows,0
472435,1.0,2.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,desktop,...,0.0,NaN,0.0,F,1.0,NaN,chrome 65.0,NaN,Windows,0
472436,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,desktop,...,0.0,0.0,0.0,T,1.0,0.0,firefox 59.0,3200x1800,Windows,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,2.0,1.0,1.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
590536,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
590537,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0
590538,1.0,1.0,1.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0


In [24]:
limpiar_dataset(data_validation, mean_values)
display(data_validation)

/home/paredesj/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,C14,C2,C11,C13,C7,C3,V192,V222,V179,DeviceType,...,V175,V333,V225,id_35,V292,V326,id_31,id_33,DeviceInfo,isFraud
472432,1.0,1.0,1.0,1.0,1.0,0.0,1.00000,1.000000,0.000000,desktop,...,0.000000,94.211435,0.000000,F,1.0,0.401879,chrome 65.0,desconocido,desconocido,1
472433,1.0,1.0,1.0,1.0,1.0,0.0,1.00000,1.000000,0.000000,mobile,...,0.000000,94.211435,0.000000,F,1.0,0.401879,chrome generic for android,desconocido,F3311,0
472434,0.0,1.0,1.0,0.0,1.0,0.0,1.00000,1.000000,0.000000,desktop,...,0.000000,94.211435,0.000000,F,1.0,0.401879,chrome 65.0,desconocido,Windows,0
472435,1.0,2.0,1.0,1.0,1.0,0.0,1.00000,2.000000,0.000000,desktop,...,0.000000,94.211435,0.000000,F,1.0,0.401879,chrome 65.0,desconocido,Windows,0
472436,1.0,1.0,1.0,1.0,0.0,0.0,1.00000,1.000000,0.000000,desktop,...,0.000000,0.000000,0.000000,T,1.0,0.000000,firefox 59.0,3200x1800,Windows,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,2.0,1.0,1.0,3.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
590536,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
590537,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0
590538,1.0,1.0,1.0,5.0,0.0,0.0,1.25468,1.367039,0.536537,desconocido,...,0.143299,94.211435,0.176914,desconocido,1.0,0.401879,desconocido,desconocido,desconocido,0


## Aplico Mean Encoding a data_train y luego a validation_train

Guardo los encodings para los features categoricos en un diccionario para luego asignarselos al set de test.

In [29]:
mean_encoding_columns = dict()

for col in data_train.columns:
    
    if(str(data_train[col].dtype) == 'object'):
       mean_encoding_columns[col]= data_train.groupby(col).isFraud.mean().to_dict()

In [54]:
mean_encoding_columns

{'DeviceType': {'desconocido': 0.021333506203599837,
  'desktop': 0.062408274512544555,
  'mobile': 0.09903889961599376},
 'M7': {'F': 0.01927899896731623,
  'T': 0.0236217096801852,
  'desconocido': 0.0439129275285019},
 'card6': {'charge card': 0.0,
  'credit': 0.06727956883236387,
  'debit': 0.023970752594735353,
  'debit or credit': 0.0,
  'desconocido': 0.028985507246376812},
 'ProductCD': {'C': 0.11329551497961354,
  'H': 0.04637769020915429,
  'R': 0.03630096574853275,
  'S': 0.06189245724129445,
  'W': 0.02069069164539987},
 'M1': {'F': 0.0,
  'T': 0.020049889798219686,
  'desconocido': 0.0499590843282486},
 'id_15': {'Found': 0.10196163355169365,
  'New': 0.04776798407733864,
  'Unknown': 0.08994599085999169,
  'desconocido': 0.021326092105708388},
 'id_29': {'Found': 0.10139587729264729,
  'NotFound': 0.04963870003203645,
  'desconocido': 0.021326092105708388},
 'M6': {'F': 0.02416746275160474,
  'T': 0.01716266708767498,
  'desconocido': 0.06833787080578439},
 'M8': {'F': 0.

In [31]:
def aplicar_mean_encoding(data_set, mean_encoding):
    
    for col in data_set.columns:
        if (str(data_set[col].dtype) == 'object'):
            data_set[col] = data_set[col].map(mean_encoding[col])

In [32]:
# Aplico el encoder al data_train
aplicar_mean_encoding(data_train, mean_encoding_columns)
display(data_train)

/home/paredesj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,C14,C2,C11,C13,C7,C3,V192,V222,V179,DeviceType,...,V175,V333,V225,id_35,V292,V326,id_31,id_33,DeviceInfo,isFraud
0,1.0,1.0,2.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
1,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
2,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
3,1.0,5.0,1.0,25.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
4,1.0,1.0,1.0,1.0,0.0,0.0,1.00000,1.000000,0.000000,0.099039,...,0.000000,0.000000,0.000000,0.043944,1.0,0.000000,0.070755,0.058296,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,1.0,2.0,1.0,2.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,2.0,0.401879,0.021362,0.033576,0.025988,0
472428,1.0,2.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,2.0,0.401879,0.021362,0.033576,0.025988,0
472429,1.0,1.0,1.0,1.0,0.0,0.0,1.00000,1.000000,0.000000,0.062408,...,0.000000,0.000000,0.000000,0.043944,1.0,0.000000,0.106546,0.024262,0.018537,0
472430,1.0,1.0,1.0,2.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0


In [34]:
# Aplico el encoder al data_validation
aplicar_mean_encoding(data_validation, mean_encoding_columns)
display(data_validation)

/home/paredesj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,C14,C2,C11,C13,C7,C3,V192,V222,V179,DeviceType,...,V175,V333,V225,id_35,V292,V326,id_31,id_33,DeviceInfo,isFraud
472432,1.0,1.0,1.0,1.0,1.0,0.0,1.00000,1.000000,0.000000,0.062408,...,0.000000,94.211435,0.000000,0.118678,1.0,0.401879,0.106546,0.033576,0.025988,1
472433,1.0,1.0,1.0,1.0,1.0,0.0,1.00000,1.000000,0.000000,0.099039,...,0.000000,94.211435,0.000000,0.118678,1.0,0.401879,0.141343,0.033576,0.000000,0
472434,0.0,1.0,1.0,0.0,1.0,0.0,1.00000,1.000000,0.000000,0.062408,...,0.000000,94.211435,0.000000,0.118678,1.0,0.401879,0.106546,0.033576,0.062120,0
472435,1.0,2.0,1.0,1.0,1.0,0.0,1.00000,2.000000,0.000000,0.062408,...,0.000000,94.211435,0.000000,0.118678,1.0,0.401879,0.106546,0.033576,0.062120,0
472436,1.0,1.0,1.0,1.0,0.0,0.0,1.00000,1.000000,0.000000,0.062408,...,0.000000,0.000000,0.000000,0.043944,1.0,0.000000,0.060345,0.000000,0.062120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,2.0,1.0,1.0,3.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
590536,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
590537,1.0,1.0,1.0,1.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0
590538,1.0,1.0,1.0,5.0,0.0,0.0,1.25468,1.367039,0.536537,0.021334,...,0.143299,94.211435,0.176914,0.021326,1.0,0.401879,0.021362,0.033576,0.025988,0


## Arreglo la data para poder usar el modelo correctamente

In [37]:
data_train = data_train.drop(columns=['isFraud'])

In [38]:
data_validation = data_validation.drop(columns=['isFraud'])

In [39]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(data_train)

data_train = scaler.fit_transform(data_train)
data_validation = scaler.transform(data_validation)

In [40]:
data_train

array([[-0.14463252, -0.09194727, -0.08779727, ..., -0.39529068,
        -0.08451211, -0.17587477],
       [-0.14463252, -0.09194727, -0.09751498, ..., -0.39529068,
        -0.08451211, -0.17587477],
       [-0.14463252, -0.09194727, -0.09751498, ..., -0.39529068,
        -0.08451211, -0.17587477],
       ...,
       [-0.14463252, -0.09194727, -0.09751498, ...,  2.04955304,
        -0.58942015, -0.31914703],
       [-0.14463252, -0.09194727, -0.09751498, ..., -0.39529068,
        -0.08451211, -0.17587477],
       [-0.14463252, -0.09194727, -0.09751498, ..., -0.39529068,
        -0.08451211, -0.17587477]])

# Creo el Modelo de Random Forest y lo entreno

Teniendo la data ordenada, creo el modelo de Random Forest y lo entreno con `data_train` y `label_train`

In [41]:
model = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [42]:
model.fit(data_train, label_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

# Metricas

## Aucroc Score

In [43]:
roc_auc_score(label_train, model.predict(data_train))

0.8540909780387488

In [44]:
roc_auc_score(label_validation, model.predict(data_validation))

0.8376671872933005

# Busqueda de Hiper-parametros

Busco hiper-parametros para `XGBoost` utilizando `GridSearchCV`

In [45]:
from scipy.stats import uniform
from sklearn.model_selection import GridSearchCV

[Hiper parametros de XGBooster](https://towardsdatascience.com/a-guide-to-xgboost-hyperparameters-87980c7f44a9)

In [46]:
hiper_parametros = dict(objective =['reg:squarederror'],
                        colsample_bytree = [0.3], 
                        learning_rate = [0.1],
                        alpha = [10],
                        max_depth = [4,8,16],
                        n_estimators = [4,8,16,32] )

In [47]:
gridSearch = GridSearchCV(model, hiper_parametros, cv=5, n_jobs=-1, scoring='roc_auc')

In [48]:
search = gridSearch.fit(data_train, label_train)

## Busco los mejores coeficientes

In [49]:
search.best_params_

{'alpha': 10,
 'colsample_bytree': 0.3,
 'learning_rate': 0.1,
 'max_depth': 16,
 'n_estimators': 32,
 'objective': 'reg:squarederror'}

In [50]:
search.best_estimator_

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=16,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=32, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

# XGBoost con los mejores hiper-parametros obtenidos 

Usando XGBoost obtengo como Hiper-Parametros obtenidos



| Hiper-Parametro   |  Mejor Valor |  
|:---:|:---:|
|  alpha | 10  |  
|  colsample_bytree |  0.3  |
|  learning_rate |  0.1  |
|  max_depth |  16|
|  n_estimators |  32 |

> Nota: El gridSearch hecho no es completamente representativo ya que XGBoost es muy pesado como para correr muchas pruebas, por eso los hiperparametros no cambian demasiado con respecto a los originalmente tomados.


Vuelvo a plantear XGBoost para ver como mejora el modelo.

In [51]:
model = search.best_estimator_

model.fit(data_train, label_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=16,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=32, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Guardo el modelo final

In [52]:
import joblib

joblib.dump(model, "random_forest_model.pkl")

['random_forest_model.pkl']

In [53]:
roc_auc_score(label_train, model.predict(data_train))

0.893217539988277

In [42]:
roc_auc_score(label_validation, model.predict(data_validation))

0.8660738148931053